# Data Initialization

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
Workspace.InitializeFrom(DataSource);

# Data Model

In [0]:
public enum Period { Previous = -1, Current = 0 };

In [0]:
public abstract record ReportParameter {
    [Display(Order = -100)]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    public int Year { get; init; }

    [Display(Order = -90)]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    public int Month { get; init; } 

    [Display(Order = -80)]
    [IdentityProperty]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }

    [IdentityProperty]
    [NotVisible]
    public Period Period { get; init; }

    [IdentityProperty]
    [NotVisible]
    [Dimension(typeof(GroupOfContract))]
    public string GroupOfContract { get; init; }
}

In [0]:
public record YieldCurveReportParameter : ReportParameter  {
    [IdentityProperty]
    [NotVisible]
    public string YieldCurveType { get; init; }

    [Dimension(typeof(Currency))]
    public string Currency { get; init; }

    public string Name { get; init; }
}

In [0]:
public record SingleDataNodeReportParameter : ReportParameter  {
    public double PremiumAllocation { get; init; }
}

In [0]:
public record InterDataNodeReportParameter : ReportParameter  {
    [Dimension(typeof(GroupOfContract), nameof(LinkedDataNode))]
    public string LinkedDataNode { get; init; }

    public double ReinsuranceCoverage { get; init; }
}

In [0]:
public record DataNodeData {
    public string DataNode { get; init; }

    //Portfolio
    [Dimension(typeof(Currency), nameof(ContractualCurrency))]
    public string ContractualCurrency { get; init; }

    [Dimension(typeof(Currency), nameof(FunctionalCurrency))]
    public string FunctionalCurrency { get; init; }

    [Dimension(typeof(LineOfBusiness))]
    public string LineOfBusiness { get; init; }

    [Dimension(typeof(ValuationApproach))]
    public string ValuationApproach { get; init; }

    public string OciType { get; init; }
    
    //GroupOfContract
    [Dimension(typeof(Portfolio))]
    public string Portfolio { get; init; }

    [Dimension(typeof(int), nameof(AnnualCohort))]
    public int AnnualCohort { get; init; }
    public string LiabilityType { get; init; }
    public string Profitability { get; init; }
    public string Partner { get; init; }
    
    //DataNodeState
    public int Year { get; init; }
    public int Month { get; init; }
    public State State { get; init; }
    public State PreviousState { get; init; }
    
    public bool IsReinsurance { get; init; }
    public DataNodeData(){}
}

# Args

In [0]:
var args = new ImportArgs("CH", 2021, 3, default, default , default);

# Data Node

In [0]:
var dataNodeData = (await Workspace.LoadDataNodesAsync(args))
    .Values
    .ToArray();
dataNodeData.Take(2)

# YieldCurve

In [0]:
var lockedYieldCurves = (await Workspace.LoadLockedInYieldCurveAsync(args, dataNodeData))
    //.Select(x => new { DataNode = x.Key, LockedIn = ( x.Value.Year, x.Value.Month, x.Value.Scenario, x.Value.Currency, x.Value.Name)})
    .Select(x => new YieldCurveReportParameter {
        GroupOfContract = x.Key,
        YieldCurveType = "Locked-In Curve",
        Year = x.Value.Year,
        Month = x.Value.Month,
        Scenario = x.Value.Scenario,
        Currency = x.Value.Currency,
        Name = x.Value.Name }
        )
    .ToArray();
lockedYieldCurves.Take(2)

In [0]:
var currentYieldCurves = (await Workspace.LoadCurrentYieldCurveAsync(args, dataNodeData))
    .Values
    //.SelectMany(x => x.Select(y => new { Period = y.Key, Current = ( y.Value.Year, y.Value.Month, y.Value.Scenario, y.Value.Currency, y.Value.Name)}))
    .SelectMany(x => x.Select(y => new YieldCurveReportParameter {
        Period = ((Period)y.Key),
        YieldCurveType = "Current Curve",
        Year = y.Value.Year,
        Month = y.Value.Month,
        Scenario = y.Value.Scenario,
        Currency = y.Value.Currency,
        Name = y.Value.Name}
        ))
    .ToArray();
currentYieldCurves.Take(2)

In [0]:
var allYieldCurves = lockedYieldCurves.Concat(lockedYieldCurves.SelectMany(loc => currentYieldCurves.Select(cuc => new YieldCurveReportParameter {
    GroupOfContract = loc.GroupOfContract,
    Period = cuc.Period,
    YieldCurveType = cuc.YieldCurveType,
    Year = cuc.Year,
    Month = cuc.Month,
    Scenario = cuc.Scenario,
    Currency = cuc.Currency,
    Name = cuc.Name}
    )))
    .ToArray();

# Single Data Node Parameters

In [0]:
var singleDataNodeParameters = (await Workspace.LoadSingleDataNodeParametersAsync(args))
    .Values
    //.SelectMany(x => x.Select(y => new { Period = y.Key, Parameter = ( y.Value.Year, y.Value.Month, y.Value.Scenario, y.Value.DataNode, y.Value.PremiumAllocation)}))
    .SelectMany(x => x.Select(y => new SingleDataNodeReportParameter {
        GroupOfContract = y.Value.DataNode,
        Period = ((Period)y.Key),
        Year = y.Value.Year,
        Month = y.Value.Month,
        Scenario = y.Value.Scenario,
        PremiumAllocation = y.Value.PremiumAllocation}
        ))
    .ToArray();
singleDataNodeParameters.Take(2)

# Inter Data Node Parameters

In [0]:
var interDataNodeParameters = (await Workspace.LoadInterDataNodeParametersAsync(args))
    .Values
    //.SelectMany(x => x.SelectMany(y => y.Value.Select(z => new { Period = y.Key,  Parameter = ( z.Year, z.Month, z.Scenario, z.DataNode, z.LinkedDataNode, z.ReinsuranceCoverage)})))
    .SelectMany(x => x.SelectMany(y => y.Value.Select(z => 
     new InterDataNodeReportParameter {
         GroupOfContract = z.DataNode,
         Period = ((Period)y.Key),
         Year = z.Year,
         Month = z.Month,
         Scenario = z.Scenario,
         LinkedDataNode = z.LinkedDataNode,
         ReinsuranceCoverage = z.ReinsuranceCoverage}
     )))
    .Distinct()// Can be removed when we get rid of the dictionary
    .SelectMany(x => new[]{x, x with { GroupOfContract = x.LinkedDataNode, LinkedDataNode = x.GroupOfContract}})
    .ToArray();
interDataNodeParameters.Take(2)

# CDR

In [0]:
var partnerRatings = (await Workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner))
    .Values
    .SelectMany(x => x.Select(y => 
    new {Period = ((Period)y.Key), y.Value}
    ));
var creditDefaultRates = (await Workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating))
    .Values
    .SelectMany(x => x.Select(y =>
    new {Period = ((Period)y.Key), y.Value}
    ));
//var partberDefaultRates

In [0]:
partnerRatings

In [0]:
creditDefaultRates

# Reports

In [0]:
await Report.ForObjects(allYieldCurves)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.YieldCurveType).GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(singleDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(interDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract).GroupRowsBy(x => x.LinkedDataNode)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()